In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow_text as tf_text
import tensorflow_hub as hub

In [2]:
df_train = pd.read_csv('train.csv')
x_train = df_train['status']
y_train = df_train['author']
author_dict = {'Justin Trudeau': 0, 'Donald J. Trump': 1}
y_train = y_train.map(lambda x: author_dict[x])

In [3]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer('https://tfhub.dev/google/electra_small/2', trainable=False)
outputs = encoder(encoder_inputs)
x = outputs['pooled_output']
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
xOut = Dense(1)(x)
model = Model(text_input, xOut)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1), optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [4]:
import numpy as np
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=6, verbose=1)
]
model.fit(np.array(x_train), np.array(y_train), batch_size=32, epochs=30, callbacks=callbacks, verbose=1, use_multiprocessing=True, validation_split=0.2)

Epoch 1/30
8/8 [==============================] - 32s 883ms/step - loss: 0.7136 - accuracy: 0.5375 - val_loss: 0.6900 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 2/30
8/8 [==============================] - 18s 3s/step - loss: 0.7429 - accuracy: 0.4292 - val_loss: 0.7523 - val_accuracy: 0.4833 - lr: 0.0010
Epoch 3/30
8/8 [==============================] - 10s 1s/step - loss: 0.7705 - accuracy: 0.4417 - val_loss: 0.7013 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 4/30
8/8 [==============================] - 10s 1s/step - loss: 0.7237 - accuracy: 0.4667 - val_loss: 0.7020 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 5/30
8/8 [==============================] - 19s 3s/step - loss: 0.7237 - accuracy: 0.4583 - val_loss: 0.7002 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 6/30
8/8 [==============================] - 1s 139ms/step - loss: 0.6986 - accuracy: 0.5000 - val_loss: 0.6947 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 7/30
8/8 [==============================] - ETA: 0s - loss: 0.7293 - accuracy: 0.45

KeyboardInterrupt: 

In [13]:
df_test = pd.read_csv('test.csv')
x_test = df_test['status']
pred = model.predict(np.array(x_test), batch_size=32, verbose=1)
pred = tf.nn.sigmoid(pred)
pred_dict = dict()
for i in range(len(x_test)):
    if pred[i] > 0.5:
        pred_dict[x_test[i]] = 'Donald J. Trump'
    else:
        pred_dict[x_test[i]] = 'Justin Trudeau'

df_submission = pd.read_csv('submission.csv')
df_submission['author'] = pred_dict.values()
df_submission.to_csv('submission_ans.csv', index=False)

4/4 [==============================] - 1s 53ms/step


In [14]:
# Connect to graders
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/alien/Documents/PyCharm Projects/Cyberthon 2021/pyctfsglib.py')
import pyctfsglib as ctfsg
import random

USER_TOKEN = "WrlLCkymxwtgFwRHZsdmKfSwcdqIpnqoXEtRkciVRZJfBJUgcEJoxVZjNTQRdqkR" # You need to fill this up
GRADER_URL = random.choice([
  "http://chals.cyberthon22t.ctf.sg:50501/",
  "http://chals.cyberthon22t.ctf.sg:50502/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)
grader.submitFile('submission_ans.csv')

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm TWEET_CLASSIFY @88cf113df561
ProofOfWork Challenge =>  ('CTFSGRB899d653b758a746cefa3b640ce03036b', 22)
ProofOfWork Answer Found! =>  775396


'{"challenge":{"name":"WhoTweetThis (challenge)"},"id":"cl22ou03fcidf0832scwqfd93","status":"PARTIALLY_CORRECT","multiplier":0.512,"submittedBy":{"username":"hci-69"},"createdAt":"2022-04-17T02:47:33Z"}'